In [ ]:
from mapping import (
    compute_area_scores
)

from alex_extractor import build_alexnet_design_matrices_with_dataloader

from plotter import plot_comparison

import pandas as pd

IMAGE_FOLDER = "../Preproc2/images/"
NUM_WORKERS = 15
METRIC = "PLS"


# Mouse-vision style parameters (for final results)
# N_BOOT = 100  # n_iter in mouse-vision
# N_SPLITS = 10  # num_splits in mouse-vision
# PLS_N_COMPONENTS = 25

N_BOOT = 1
N_SPLITS = 1
PLS_N_COMPONENTS = 25

# Passing the stimuli to AlexNet and extracting activations

The idea is to use a pretrained model (AlexNet) to extract features from the stimuli images, and then use these features to predict neural responses. 

Using the formalism discussed in the [neural_to_neural.ipynb](neural_to_neural.ipynb) notebook, we can think of the model as a "source" and the neural data as a "target". We will compute the similarity between the model's representation and the neural representation using the same metrics as before (RSA, CKA, PLS).

Here the design matrices of the AlexNet representation do not have trials, to the splits (required for the reliability computation in the corrected similarity version) are performed only on the neural data.

In [2]:
index_model_random = build_alexnet_design_matrices_with_dataloader(
    folder=IMAGE_FOLDER,
    batch_size=16,
    shuffle=False,
    num_workers=NUM_WORKERS,
    weights="random",
    device="cuda",
    amp=True,
    layers_keep=["conv1", "conv2", "conv3", "conv4", "conv5"],
    save_dir="alex-random-matrices",
    return_in_memory=False
)

In [3]:
index_model_inet_no_diet = build_alexnet_design_matrices_with_dataloader(
    folder=IMAGE_FOLDER,
    batch_size=16,
    shuffle=False,
    num_workers=NUM_WORKERS,
    weights="checkpoints/supervised_no-diet/20250915_023037_sgd/checkpoint_epoch_60.pth",
    device="cuda",
    amp=True,
    layers_keep=["conv1", "conv2", "conv3", "conv4", "conv5"],
    save_dir="alex-matrices-supervised-no-diet",
    return_in_memory=False
)

In [4]:
index_model_inet_diet = build_alexnet_design_matrices_with_dataloader(
    folder=IMAGE_FOLDER,
    batch_size=16,
    shuffle=False,
    num_workers=NUM_WORKERS,
    weights="checkpoints/supervised_diet/20250930_022201_no-aug/checkpoint_epoch_60.pth",
    device="cuda",
    amp=True,
    layers_keep=["conv1", "conv2", "conv3", "conv4", "conv5"],
    save_dir="alex-matrices-supervised-diet",
    return_in_memory=False
)

In [5]:
index_model_inet_ss_no_diet = build_alexnet_design_matrices_with_dataloader(
    folder=IMAGE_FOLDER,
    batch_size=16,
    shuffle=False,
    num_workers=NUM_WORKERS,
    weights="checkpoints/self-supervised_no-diet/20250922_144504_warmup+dynamic/checkpoint_epoch_60.pth",
    device="cuda",
    amp=True,
    layers_keep=["conv1", "conv2", "conv3", "conv4", "conv5"],
    save_dir="alex-matrices-self-supervised-no-diet",
    return_in_memory=False
)

In [6]:
index_model_inet_ss_diet = build_alexnet_design_matrices_with_dataloader(
    folder=IMAGE_FOLDER,
    batch_size=16,
    shuffle=False,
    num_workers=NUM_WORKERS,
    weights="checkpoints/self-supervised_diet/20250929_010742_warmup+dynamic/checkpoint_epoch_60.pth",
    device="cuda",
    amp=True,
    layers_keep=["conv1", "conv2", "conv3", "conv4", "conv5"],
    save_dir="alex-matrices-self-supervised-diet",
    return_in_memory=False
)

Loading the index file with paths to neural data

In [7]:
index_df = pd.read_csv('../Preproc2/data/combined_index.csv')

In [ ]:
# random
layer_scores_random, median_scores_random = compute_area_scores(
    index_model_random,
    index_df,
    verbose=True,
    n_boot=N_BOOT,
    n_splits=N_SPLITS,
    n_components=PLS_N_COMPONENTS,
    model_name='random',
    test_areas=["VISp"],
)

Filtered areas: ['VISp']
Layer: conv1, Area: VISp, Specimen: 717038288, Score: 0.2230
Layer: conv1, Area: VISp, Specimen: 719828690, Score: 0.1648
Layer: conv1, Area: VISp, Specimen: 730756780, Score: 0.1763
Layer: conv1, Area: VISp, Specimen: 738651054, Score: 0.2002
Layer: conv1, Area: VISp, Specimen: 739783171, Score: 0.1950
Layer: conv1, Area: VISp, Specimen: 742602892, Score: 0.2077
Layer: conv1, Area: VISp, Specimen: 744915204, Score: 0.2624
Layer: conv1, Area: VISp, Specimen: 772616823, Score: 0.2064
Filtered areas: ['VISp']


/home/spina/Desktop/units/DL/project/mice-representation-clean/representation/neural_maps.py:189: RuntimeWarning: invalid value encountered in sqrt
  valid_mask &= (denom > 0)
/home/spina/Desktop/units/DL/project/mice-representation-clean/representation/neural_maps.py:206: RuntimeWarning: Mean of empty slice
  # POOLED SOURCE → target B : PLS (paper-style corrected)


Layer: conv2, Area: VISp, Specimen: 717038288, Score: 0.2795
Layer: conv2, Area: VISp, Specimen: 719828690, Score: 0.2457
Layer: conv2, Area: VISp, Specimen: 730756780, Score: 0.2861


In [ ]:
# inet no diet
layer_scores_inet_no_diet, median_scores_inet_no_diet = compute_area_scores(
    index_model_inet_no_diet,
    index_df,
    verbose=True,
    n_boot=N_BOOT,
    n_splits=N_SPLITS,
    n_components=PLS_N_COMPONENTS,
    model_name='supervised_no-diet'
)

In [ ]:
# inet diet
layer_scores_inet_diet, median_scores_inet_diet = compute_area_scores(
    index_model_inet_diet,
    index_df,
    verbose=True,
    n_boot=N_BOOT,
    n_splits=N_SPLITS,
    n_components=PLS_N_COMPONENTS,
    model_name='supervised_diet'
)

In [ ]:
# inet ss no diet
layer_scores_inet_ss_no_diet, median_scores_inet_ss_no_diet = compute_area_scores(
    index_model_inet_ss_no_diet,
    index_df,
    sim_metric=METRIC,
    verbose=True,
    n_boot=N_BOOT,
    n_splits=N_SPLITS,
    n_components=PLS_N_COMPONENTS,
    model_name='self-supervised_no-diet'
)

In [ ]:
# inet ss diet
layer_scores_inet_ss_diet, median_scores_inet_ss_diet = compute_area_scores(
    index_model_inet_ss_diet,
    index_df,
    sim_metric=METRIC,
    verbose=True,
    n_boot=N_BOOT,
    n_splits=N_SPLITS,
    n_components=PLS_N_COMPONENTS,
    model_name='self-supervised_diet'
)

In [ ]:
from plotter import plot_comparison_multi

# plot_comparison_multi([median_scores_random, median_scores_inet_no_diet, median_scores_inet_diet, median_scores_inet_ss_no_diet, median_scores_inet_ss_diet], ["random", "supervised_no-diet", "supervised_diet", "self-supervised_no-diet", "self-supervised_diet"], metric_name=METRIC)
plot_comparison_multi([median_scores_random], ["random"], metric_name=METRIC)

NameError: name 'median_scores_random' is not defined